In [1]:
!pip install elasticsearch==7.0.1
!pip install -q openai-whisper ffmpeg
!apt-get update
!apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 24.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 7.9 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,196 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubu

In [2]:
import locale
import whisper
import torch
from moviepy.editor import VideoFileClip
import os
import cv2
from elasticsearch import Elasticsearch
import json

In [3]:
# Check if CUDA is available, otherwise use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the Whisper model on the appropriate device
model = whisper.load_model("medium", device=device)

100%|█████████████████████████████████████| 1.42G/1.42G [00:17<00:00, 88.0MiB/s]


In [4]:
def save_transcription(transcription_data, output_file):
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    # If the file exists, load existing data
    if os.path.exists(output_file):
        try:
            with open(output_file, 'r', encoding='utf-8') as f:
                existing_data = json.load(f)
        except json.JSONDecodeError:
            existing_data = []
    else:
        existing_data = []

    # Add new transcription data to existing data
    existing_data.extend(transcription_data)

    # Save the updated data to the output file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)

def transcribe_audio(video_id, audio_path, fps, output_file, frame_sampling_rate=10):
    """
    Transcribes audio and aligns it with sampled frames.
    
    Parameters:
        video_id (str): ID of the video.
        audio_path (str): Path to the audio file.
        fps (int): Frames per second of the video.
        output_file (str): Path to save the output data.
        frame_sampling_rate (int): Interval for sampling frames (default: every 10 frames).
    
    Returns:
        list: Transcription data aligned with sampled frames.
    """
    # Transcribe the audio file
    result = model.transcribe(audio_path)
    print(result)
    # Prepare the list to store transcription data
    transcription_data = []

    # Process each transcription segment
    for segment in result['segments']:
        start_time = segment['start']
        end_time = segment['end']
        text = segment['text']

        # Calculate frame range based on time and FPS
        start_frame = int(start_time * fps)
        end_frame = int(end_time * fps)

        # Filter sampled frames within the range
        sampled_frames = [
            frame
            for frame in range(start_frame, end_frame + 1)
            if frame % frame_sampling_rate == 0
        ]

        # If no sampled frames are within the range, skip this segment
        if not sampled_frames:
            continue

        video_folder = f"Videos_{video_id.split('_')[0]}"

        # Prepare a dictionary to store this segment's data
        segment_data = {
            "video_folder": video_folder,
            "video_id": video_id,
            "start_frame": start_frame,
            "end_frame": end_frame,
            "fps": fps,
            "text": text,
            "sampled_frames": sampled_frames
        }

        # Append the segment data to the list
        transcription_data.append(segment_data)

    # Save transcription data to output file
    with open(output_file, "w") as f:
        json.dump(transcription_data, f, ensure_ascii=False, indent=4)

    return transcription_data

def extract_audio_from_video(video_path, output_folder, file):

    # Tạo đường dẫn file âm thanh với tên file.wav
    output_audio_path = os.path.join(output_folder, f"{file}.wav")

    # Load video file
    video = VideoFileClip(video_path)

    # Extract audio
    audio = video.audio

    # Save the audio file
    audio.write_audiofile(output_audio_path)

    # Close the video clip
    video.close()

def calculate_fps(video_path):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get the frame rate (fps) using the CAP_PROP_FPS property
    fps = video.get(cv2.CAP_PROP_FPS)

    # Release the video capture object
    video.release()

    return fps

def load_checkpoint(CHECKPOINT_FILE):
    # Kiểm tra nếu tệp checkpoint cho thư mục này đã tồn tại
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
            try:
                return set(json.load(f))  # Trả về tập hợp các tệp đã xử lý
            except json.JSONDecodeError:
                return set()  # Nếu lỗi, trả về tập rỗng
    return set()  # Nếu không có tệp checkpoint, trả về tập rỗng

def update_checkpoint(processed_files, CHECKPOINT_FILE):
    # Kiểm tra nếu tệp checkpoint đã tồn tại và nạp dữ liệu cũ
    existing_data = load_checkpoint(CHECKPOINT_FILE)

    # Cập nhật danh sách tệp đã xử lý
    updated_data = list(existing_data | processed_files)

    # Ghi dữ liệu cập nhật vào checkpoint file
    with open(CHECKPOINT_FILE, 'w', encoding='utf-8') as f:
        json.dump(updated_data, f, ensure_ascii=False, indent=4)

def process_video_range(base_path, CHECKPOINT_FOLDER, output_folder):
    folder_name = f"Videos_L01"
     # Đường dẫn checkpoint cho từng folder
    CHECKPOINT_FILE = os.path.join(CHECKPOINT_FOLDER, f"checkpoint_L01.json")
    if os.path.exists(CHECKPOINT_FOLDER):
        os.makedirs(CHECKPOINT_FOLDER, exist_ok=True)
    if os.path.exists(output_folder):
        os.makedirs(output_folder, exist_ok=True)
    # Tải checkpoint cho folder hiện tại
    processed_files = load_checkpoint(CHECKPOINT_FILE)

    if os.path.isdir(base_path):
        for audio_file in os.listdir(base_path):
            print(audio_file)
            audio_id = os.path.splitext(audio_file)[0]
            audio_file_name = f"{audio_id}.wav"
            audio_path = os.path.join(base_path, audio_file_name)
            output_file = os.path.join(output_folder, f"{audio_id}_transcription.json")

            # Kiểm tra xem âm thanh đã được xử lý chưa
            if audio_file_name in processed_files:
                print(f"Skipping already processed audio: {audio_file_name}")
                continue

            # Calculate FPS
            fps = 25

            # Transcribe audio
            transcription_data = transcribe_audio(audio_id, audio_path, fps, output_file)
            print(f"Transcribed audio for {audio_id}")

            # Save transcription data
            save_transcription(transcription_data, output_file)
            print(f"Saved transcription data for {audio_file}")

            # Cập nhật danh sách tệp đã xử lý
            processed_files.add(audio_file_name)
            update_checkpoint(processed_files, CHECKPOINT_FILE)
            print(f"Updated checkpoint for {audio_file}")
    print(f"Finished processing folder {folder_name}")

In [5]:
"""### Extract audio"""

# path = "/kaggle/input/aic-2024-all-video-448p-b1/All_Videos_448p/All_Videos_448p"
# output = '/kaggle/working/audio'
# if not os.path.exists(output):
#     os.makedirs(output)

# for file in os.listdir(path):
#     if file.endswith(".mp4"):
#         video_path = os.path.join(path, file)

#         file_name = file.split('.')[0]
#         if file_name.split('_')[0] == 'L01': 
#             extract_audio_from_video(video_path, output, file_name)
#             print(f"Extracted audio from video: {file}")

'### Extract audio'

In [6]:
%cp /kaggle/input/asr-output/asr -r /kaggle/working/

In [7]:
"""### Run transcribe audio"""
CHECKPOINT_FOLDER = "/kaggle/working/asr/checkpoints"
audio_path = "/kaggle/input/audio-video1/audio"
output_folder = "/kaggle/working/asr/output"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(CHECKPOINT_FOLDER, exist_ok=True)

process_video_range(audio_path, CHECKPOINT_FOLDER, output_folder)

L01_V001.wav
Skipping already processed audio: L01_V001.wav
L01_V002.wav
Skipping already processed audio: L01_V002.wav
L01_V026.wav
Skipping already processed audio: L01_V026.wav
L01_V024.wav
Skipping already processed audio: L01_V024.wav
L01_V025.wav
Skipping already processed audio: L01_V025.wav
L01_V029.wav
{'text': ' Chào quý vị, chúng tôi rất vui được gặp lại quý vị trong chương trình 60 giây của Đài Truyền Hình Thành phố Hồ Chí Minh. 60 giây hôm nay sẽ mang đến cho quý vị các thông tin được quan tâm nhất trong ngày. Thành phố Hồ Chí Minh khám chữa bệnh bị phí cho toàn bộ người cao tuổi. Các nạn nhân của Tân Hoàng Bình, Du Khán Nam sẽ được nhận lại tiền. Một trẻ em tử vong nghi ngộ độc bắn được phát trong đêm hội trung thuộc. Thưa quý vị, Sở Y tế Thành phố Hồ Chí Minh cho biết hưởng ứng tháng thành động quốc gia về người cao tuổi Việt Nam năm 2023. Trong tháng 10, Thành phố Hồ Chí Minh tổ chức thăm khám, chữa bệnh bị phí cho toàn bộ người cao tuổi trên địa bàn Thành phố. Trong su